# 211220 Ground truth
- 211102 synapse annotation
- 211119 synapse annotation

In [1]:
import numpy as np
import tifffile as tif

from cloudvolume import CloudVolume

import itertools
import operator

In [2]:
def chunk_bboxes_2d(vol_size, chunk_size, overlap=(0, 0), offset=None, mip=0):

    if mip > 0:
        mip_factor = 2 ** mip
        vol_size = (vol_size[0]//mip_factor,
                    vol_size[1]//mip_factor)

        chunk_size = (chunk_size[0]//mip_factor,
                      chunk_size[1]//mip_factor)

        overlap = (overlap[0]//mip_factor,
                   overlap[1]//mip_factor)

        if offset is not None:
            offset = (offset[0]//mip_factor,
                      offset[1]//mip_factor)

    x_bnds = bounds1D_overlap(vol_size[0], chunk_size[0], overlap[0])
    y_bnds = bounds1D_overlap(vol_size[1], chunk_size[1], overlap[1])
    
    bboxes = [tuple(zip(xs, ys))
              for (xs, ys) in itertools.product(x_bnds, y_bnds)]

    if offset is not None:
        bboxes = [(tuple(map(operator.add, bb[0], offset)),
                   tuple(map(operator.add, bb[1], offset)))
                  for bb in bboxes]

    return bboxes


# Boundaries with overlap 1D
def bounds1D_overlap(full_width, step_size, overlap=0):

    assert step_size > 0, "invalid step_size: {}".format(step_size)
    assert full_width > 0, "invalid volume_width: {}".format(full_width)
    assert overlap >= 0, "invalid overlap: {}".format(overlap)

    start = 0
    end = step_size

    bounds = []
    while end < full_width:
        bounds.append((start, end))

        start += step_size - overlap
        end = start + step_size

#     # last window
#     end = full_width
#     bounds.append((start, end))

    return bounds

## EM image

In [3]:
cvname = "gs://gnd-dauer1/rough_align"

In [4]:
vol = CloudVolume(cvname, mip=1, parallel=True, progress=False)

In [5]:
img_size = (8000,8000)
chunk_size = (320,320)
overlap_size = (32,32)
offset = (6000,6000)

bbox_list = chunk_bboxes_2d(img_size, chunk_size, overlap_size, offset=offset)

In [34]:
nsec = 50
zoff = 191

img_list = []
for i in range(nsec):
    
    zidx = zoff + i
    
    img = vol[:,:,zidx][:,:,0,0]
    
    for b in bbox_list:
        
        img_sample = img[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        img_list.append(img_sample)
        
    print("Section {} loaded.".format(zidx))

Section 191 loaded.
Section 192 loaded.
Section 193 loaded.
Section 194 loaded.
Section 195 loaded.
Section 196 loaded.
Section 197 loaded.
Section 198 loaded.
Section 199 loaded.
Section 200 loaded.
Section 201 loaded.
Section 202 loaded.
Section 203 loaded.
Section 204 loaded.
Section 205 loaded.
Section 206 loaded.
Section 207 loaded.
Section 208 loaded.
Section 209 loaded.
Section 210 loaded.
Section 211 loaded.
Section 212 loaded.
Section 213 loaded.
Section 214 loaded.
Section 215 loaded.
Section 216 loaded.
Section 217 loaded.
Section 218 loaded.
Section 219 loaded.
Section 220 loaded.
Section 221 loaded.
Section 222 loaded.
Section 223 loaded.
Section 224 loaded.
Section 225 loaded.
Section 226 loaded.
Section 227 loaded.
Section 228 loaded.
Section 229 loaded.
Section 230 loaded.
Section 231 loaded.
Section 232 loaded.
Section 233 loaded.
Section 234 loaded.
Section 235 loaded.
Section 236 loaded.
Section 237 loaded.
Section 238 loaded.
Section 239 loaded.
Section 240 loaded.


In [24]:
nsec = 10
zoff = 371

img_list = []
for i in range(nsec):
    
    zidx = zoff + i
    
    img = vol[:,:,zidx][:,:,0,0]
    
    for b in bbox_list:
        
        img_sample = img[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        img_list.append(img_sample)
        
    print("Section {} loaded.".format(zidx))

Section 371 loaded.
Section 372 loaded.
Section 373 loaded.
Section 374 loaded.
Section 375 loaded.
Section 376 loaded.
Section 377 loaded.
Section 378 loaded.
Section 379 loaded.
Section 380 loaded.


In [25]:
for i in range(len(img_list)):
    
    img_list[i] = img_list[i].reshape((320,320,1))

In [26]:
img_stack = np.concatenate(img_list, axis=2)

In [27]:
tif.imwrite("/home/jabae/research/Cnapse/groundtruth/img_211119_vol5.tif", img_stack)

## 211102 synapse annotation

In [3]:
cvname1 = "gs://gnd-dauer1/syn_label/211102_syn_label_hs"
cvname2 = "gs://gnd-dauer1/syn_label/211102_syn_label_th"

In [4]:
vol1 = CloudVolume(cvname1, parallel=True, progress=False)
vol2 = CloudVolume(cvname2, parallel=True, progress=False)

In [8]:
img_size = (8000,8000)
chunk_size = (320,320)
overlap_size = (32,32)
offset = (6000,6000)

bbox_list = chunk_bboxes_2d(img_size, chunk_size, overlap_size, offset=offset)

In [ ]:
nsec = 50
zoff = 191

lab_list = []
for i in range(nsec):
    
    zidx = zoff + i
    
    lab1 = vol1[:,:,zidx][:,:,0,0]
    lab2 = vol2[:,:,zidx][:,:,0,0]
    
    lab1 = (lab1>0).astype("uint8")
    lab2 = (lab2>0).astype("uint8")
    
    for b in bbox_list:
        
        lab1_sample = lab1[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        lab2_sample = lab2[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        
        lab_sample = lab1_sample + lab2_sample
        lab_sample = (lab_sample>0).astype("uint8")
        lab_sample = lab_sample.reshape(lab_sample.shape+(1,))
        
        lab_list.append(lab_sample)
        
    print("Section {} loaded.".format(zidx))

In [14]:
lab_stack = np.concatenate(lab_list, axis=2)

In [15]:
tif.imwrite("/home/jabae/research/Cnapse/lab_211102_vol2.tif", lab_stack)

## 211119 synapse annotation

In [17]:
cvname1 = "gs://gnd-dauer1/syn_label/211119_syn_label_hs"
cvname2 = "gs://gnd-dauer1/syn_label/211119_syn_label_th"

In [18]:
vol1 = CloudVolume(cvname1, parallel=True, progress=False)
vol2 = CloudVolume(cvname2, parallel=True, progress=False)

In [19]:
img_size = (8000,8000)
chunk_size = (320,320)
overlap_size = (32,32)
offset = (6000,6000)

bbox_list = chunk_bboxes_2d(img_size, chunk_size, overlap_size, offset=offset)

In [20]:
nsec = 50
zoff = 331

lab_list = []
for i in range(nsec):
    
    zidx = zoff + i
    
    lab1 = vol1[:,:,zidx][:,:,0,0]
    lab2 = vol2[:,:,zidx][:,:,0,0]
    
    lab1 = (lab1>0).astype("uint8")
    lab2 = (lab2>0).astype("uint8")
    
    for b in bbox_list:
        
        lab1_sample = lab1[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        lab2_sample = lab2[b[0][0]:b[1][0],b[0][1]:b[1][1]]
        
        lab_sample = lab1_sample + lab2_sample
        lab_sample = (lab_sample>0).astype("uint8")
        lab_sample = lab_sample.reshape(lab_sample.shape+(1,))
        
        lab_list.append(lab_sample)
        
    print("Section {} loaded.".format(zidx))

Section 331 loaded.
Section 332 loaded.
Section 333 loaded.
Section 334 loaded.
Section 335 loaded.
Section 336 loaded.
Section 337 loaded.
Section 338 loaded.
Section 339 loaded.
Section 340 loaded.
Section 341 loaded.
Section 342 loaded.
Section 343 loaded.
Section 344 loaded.
Section 345 loaded.
Section 346 loaded.
Section 347 loaded.
Section 348 loaded.
Section 349 loaded.
Section 350 loaded.
Section 351 loaded.
Section 352 loaded.
Section 353 loaded.
Section 354 loaded.
Section 355 loaded.
Section 356 loaded.
Section 357 loaded.
Section 358 loaded.
Section 359 loaded.
Section 360 loaded.
Section 361 loaded.
Section 362 loaded.
Section 363 loaded.
Section 364 loaded.
Section 365 loaded.
Section 366 loaded.
Section 367 loaded.
Section 368 loaded.
Section 369 loaded.
Section 370 loaded.
Section 371 loaded.
Section 372 loaded.
Section 373 loaded.
Section 374 loaded.
Section 375 loaded.
Section 376 loaded.
Section 377 loaded.
Section 378 loaded.
Section 379 loaded.
Section 380 loaded.


In [21]:
lab_stack = np.concatenate(lab_list, axis=2)

In [22]:
tif.imwrite("/home/jabae/research/Cnapse/lab_211119.tif", lab_stack)